In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [3]:
#Reading the data
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
data=data.drop('Surname',axis=1)
data=data.drop('RowNumber',axis=1)
data=data.drop('CustomerId',axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [5]:
#Preprocessing the data
label_en_gender=LabelEncoder()
data['Gender']=label_en_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [6]:
#One Hot Encoding
from sklearn.preprocessing import OneHotEncoder
OneHot_encoder_geo=OneHotEncoder()
encoded_value=OneHot_encoder_geo.fit_transform(data[['Geography']])
encoded_value

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [7]:
OneHot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [8]:
#Converting to dataframe
geo_encoded_df=pd.DataFrame(encoded_value.toarray(),
                columns=OneHot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [9]:
#Combine the one hot encoded column with the data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
#Saving the encoders
with open('label_en_gender.pkl','wb') as file:
    pickle.dump(label_en_gender,file)
with open('OneHot_encoder_geo.pkl','wb') as file:
    pickle.dump(OneHot_encoder_geo,file)


In [11]:
#Dividing the data into dependent and independent features
y=data['Exited']
x=data.drop('Exited',axis=1)


#Training and Testing data
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.2,random_state=42)

#Scaling the data
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [12]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

## ANN IMPLEMENTATION

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [14]:
(x_train.shape[1],)

(12,)

In [15]:
#Create ANN model
model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)),   ##HL1
    Dense(32,activation='relu'),                                   ##HL2
    Dense(1,activation='sigmoid')                                  ##OP L
])

In [16]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
#Compile the model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [19]:
#Setting the tensorboard
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [20]:
#Setup the earlystopping model
early_stopping_model=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [21]:
#Train the model
history=model.fit(
    x_train,y_train,validation_data=(x_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_model]
)

Epoch 1/100


63/63 [==============================] - 10s 104ms/step - loss: 0.4633 - accuracy: 0.8070 - val_loss: 0.4157 - val_accuracy: 0.8255
Epoch 2/100
63/63 [==============================] - 4s 66ms/step - loss: 0.3922 - accuracy: 0.8395 - val_loss: 0.3768 - val_accuracy: 0.8441
Epoch 3/100
63/63 [==============================] - 4s 57ms/step - loss: 0.3651 - accuracy: 0.8500 - val_loss: 0.3754 - val_accuracy: 0.8456
Epoch 4/100
63/63 [==============================] - 2s 39ms/step - loss: 0.3434 - accuracy: 0.8580 - val_loss: 0.3614 - val_accuracy: 0.8514
Epoch 5/100
63/63 [==============================] - 2s 34ms/step - loss: 0.3364 - accuracy: 0.8575 - val_loss: 0.3758 - val_accuracy: 0.8435
Epoch 6/100
63/63 [==============================] - 1s 22ms/step - loss: 0.3281 - accuracy: 0.8660 - val_loss: 0.3658 - val_accuracy: 0.8524
Epoch 7/100
63/63 [==============================] - 3s 49ms/step - loss: 0.3215 - accuracy: 0.8695 - val_loss: 0.3702 - val_accuracy: 0.8479
Ep

In [22]:
model.save('model.h5')

d:\NewDownloads\ANN Classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [23]:
#Load the tensorboard extension
#%load_ext tensorboard

In [24]:
%load_ext tensorboard

In [27]:
%tensorboard --logdir logs/fit20250719+234621

Reusing TensorBoard on port 6007 (pid 14788), started 2 days, 12:27:09 ago. (Use '!kill 14788' to kill it.)

In [26]:
import tensorflow as tf
print(tf.__version__)


2.15.0
